# 🦜🔗 Multimodale LLMs (Vision)


In [ ]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.schema import StrOutputParser, HumanMessage, SystemMessage, BaseMessage
from langchain.prompts import (
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from helpers import llm

#### Bilder malen mit Dall-E


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        HumanMessagePromptTemplate.from_template(
            "Generate a short prompt to generate an image based on the following description: {image_desc}"
        )
    ]
)
chain = prompt | llm(temperature=0.9) | StrOutputParser()
instruction = chain.invoke({"image_desc": "halloween night at a haunted museum"})
print(instruction)

In [ ]:
image_url = DallEAPIWrapper(model="dall-e-2", size="1024x1024").run(instruction)
print(str(image_url))

#### Bilder beschreiben mit gpt-4o


In [ ]:
vision_llm = llm(model="gpt-4o", max_tokens=1024)

# OpenAI Format für multimodale Inputs
input = [
    HumanMessage(
        content=[
            "What's in this image?",
            {"image_url": "https://joscha.com/data/media/cartoons/130608.png"},
        ]
    )
]


response = vision_llm.invoke(input)
print(response.content)

## ✅ Aufgabe

#### Man kann so etwas natürlich auch als Tool in einem Agenten einsetzen


In [ ]:
import base64


def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
# from langchain_core.tools


# @...
def analyze_image(image_path: str, question: str) -> str:
    """This tool can extract general information from an image given a query."""
    # base64_image =
    vision_llm_input = [
        HumanMessage(
            content=[
                question,
                {"image_url": f"data:image/jpeg;base64,{base64_image}"},
            ]
        )
    ]
    # return...


tools = [analyze_image]

In [ ]:
from langgraph.prebuilt import create_react_agent  # noqa

system_message = SystemMessage(content="Du bist ein hilfsbereiter Assistent.")
# agent_executor =

In [ ]:
input = {
    "messages": [
        HumanMessage(
            content="Mein Ticket ist hier: DBTicket.jpg. Wann sollte ich wo am Bahnhof sein?"
        )
    ]
}

for event in agent_executor.stream(input, stream_mode="values"):
    message: BaseMessage = event["messages"][-1]
    message.pretty_print()

#### Persönliche Daten?


In [ ]:
input = {
    "messages": [
        HumanMessage(
            content="Mein Ticket ist hier: DBTicket.jpg. Ich kann schlecht lesen, für wen ist das Ticket?"
        )
    ]
}

for event in agent_executor.stream(input, stream_mode="values"):
    message: BaseMessage = event["messages"][-1]
    message.pretty_print()